In [ ]:
train_dir = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train'
test_dir = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test'
train_csv_path = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train_labels.csv'
test_ids_path = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv'
sample_submission_path = '/kaggle/input/soil-classification-part-2/soil_competition-2025/sample_submission.csv'

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
pseudo_negatives = torch.utils.data.Subset(cifar10, range(0, 2000))

In [ ]:
train_dataset = SoilDataset(train_df, train_dir, train_transform)
train_dataset = ConcatDataset([train_dataset, pseudo_negatives])

In [ ]:
val_dataset = SoilDataset(val_df, train_dir, val_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Sequential(nn.Linear(model.classifier[1].in_features, 1), nn.Sigmoid())
model.to(device)

In [ ]:
def train_model(model, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(device)
            labels = labels.to(device).unsqueeze(1)

In [ ]:
optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
train_model(model, epochs=10)
evaluate_model(model, val_loader)